In [6]:
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
service = Service("chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.get("https://www.flipkart.com/search?q=android+mobile")
time.sleep(5)
html=driver.page_source
soup = BeautifulSoup(html,'html.parser')
driver.quit()
total_pages=soup.find('div',class_="_1G0WLw").find('span').text.split(" ")
first_page=int(total_pages[1])
last_page=int(total_pages[3])


  

In [8]:
from tqdm import tqdm  # Import tqdm for progress bar

data = []
for i in tqdm(range(first_page, last_page), desc="Scraping pages", unit="page"):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")  
    chrome_options.add_argument("--no-sandbox") 

    service = Service("chromedriver.exe")
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    driver.get(f"https://www.flipkart.com/search?q=android+mobile&page={i}")
    time.sleep(2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    driver.quit()    
    for phn in tqdm(range(0, len(soup.find_all('div', class_="tUxRFH"))), desc="Scraping phones", unit="phone"):
        phn_soup = soup.find_all('div', class_="tUxRFH")[phn]
        
        try:
            title = phn_soup.find('div', class_="KzDlHZ").text
        except AttributeError:
            title = np.nan
            
        try:
            price_without_dis = phn_soup.find('div', class_="yRaY8j ZYYwLA").text
        except AttributeError:
            price_without_dis = np.nan
            
        try:
            price_with_dis = phn_soup.find('div', class_="Nx9bqj _4b5DiR").text
        except AttributeError:
            price_with_dis = np.nan
            
        try:
            li_items = phn_soup.find_all('li', class_="J+igdf")
            ram = li_items[0].text.split("|")[0].strip() if len(li_items) > 0 else np.nan
            rom = li_items[0].text.split("|")[1].strip() if len(li_items) > 0 else np.nan
            display = li_items[1].text.strip() if len(li_items) > 1 else np.nan
            front_cam = li_items[2].text.split("|")[0].strip() if len(li_items) > 2 else np.nan
            back_cam = li_items[2].text.split("|")[1].strip() if len(li_items) > 2 else np.nan
            battery = li_items[3].text.strip() if len(li_items) > 3 else np.nan
            processor = li_items[4].text.strip() if len(li_items) > 4 else np.nan
            warranty = li_items[4].text.split(" ")[0] if len(li_items) > 4 else np.nan
        except (IndexError, AttributeError):
            ram, rom, display, front_cam, back_cam, battery, processor, warranty = [np.nan] * 8
        
        data.append({
            "title": title,
            "price_without_dis": price_without_dis,
            "price_with_dis": price_with_dis,
            "ram": ram,
            "rom": rom,
            "display": display,
            "front_cam": front_cam,
            "back_cam": back_cam,
            "battery": battery,
            "processor": processor,
            "warranty": warranty
        })


Scraping pages:  14%|█▍        | 41/283 [08:50<1:08:48, 17.06s/page]
Scraping phones: 0phone [00:00, ?phone/s]
Scraping pages:  15%|█▍        | 42/283 [08:57<56:19, 14.02s/page]  
Scraping phones: 0phone [00:00, ?phone/s]
Scraping pages:  15%|█▌        | 43/283 [09:04<47:24, 11.85s/page]
Scraping phones: 0phone [00:00, ?phone/s]
Scraping pages:  16%|█▌        | 44/283 [09:19<50:55, 12.78s/page]
Scraping phones: 0phone [00:00, ?phone/s]
Scraping pages:  16%|█▌        | 45/283 [09:25<43:33, 10.98s/page]
Scraping phones: 0phone [00:00, ?phone/s]
Scraping pages:  16%|█▋        | 46/283 [09:36<43:15, 10.95s/page]
Scraping phones: 0phone [00:00, ?phone/s]
Scraping pages:  17%|█▋        | 47/283 [09:43<38:10,  9.71s/page]
Scraping phones: 0phone [00:00, ?phone/s]
Scraping pages:  17%|█▋        | 48/283 [09:52<36:49,  9.40s/page]
Scraping phones: 0phone [00:00, ?phone/s]
Scraping pages:  17%|█▋        | 49/283 [10:07<43:01, 11.03s/page]
Scraping phones: 0phone [00:00, ?phone/s]
Scraping pages:

In [14]:
data_df = pdDataFrame(data)


In [42]:
data_df

,title,price_without_dis,price_with_dis,ram,rom,display,front_cam,back_cam,battery,processor
0,"SAMSUNG Galaxy F05 (Twilight Blue, 64 GB)","₹9,999","₹6,299",4 GB RAM,64 GB ROM,17.12 cm (6.74 inch) HD+ Display,50MP + 2MP,8MP Front Camera,5000 mAh Battery,Helio G85 Processor
1,"POCO C61 (Ethereal Blue, 64 GB)","₹8,999","₹5,899",4 GB RAM,64 GB ROM,17.04 cm (6.71 inch) HD+ Display,8MP Rear Camera,5MP Front Camera,5000 mAh Battery,Helio G36 Processor
2,"MOTOROLA g05 (Forest Green, 64 GB)","₹9,999","₹6,999",4 GB RAM,64 GB ROM,16.94 cm (6.67 inch) HD+ Display,50MP Rear Camera,8MP Front Camera,5200 mAh Battery,Helio G81 Processor
3,POCO C61 - Locked with Airtel Prepaid (Diamon...,"₹8,999","₹5,399",4 GB RAM,64 GB ROM,17.04 cm (6.71 inch) HD+ Display,8MP Rear Camera,5MP Front Camera,5000 mAh Battery,Helio G36 Processor
4,"POCO C61 (Diamond Dust Black, 64 GB)","₹8,999","₹5,899",4 GB RAM,64 GB ROM,17.04 cm (6.71 inch) HD+ Display,8MP Rear Camera,5MP Front Camera,5000 mAh Battery,Helio G36 Processor
...,...,...,...,...,...,...,...,...,...,...
979,"REDMI Note-14 Pro+ 5G (Phantom Purple, 128 GB)","₹34,999","₹34,499",8 GB RAM,128 GB ROM,16.94 cm (6.67 inch) Display,50MP + 50MP + 8MP,20MP Front Camera,6200 mAh Battery,7s Gen 3 Processor
980,"Honor 200 5G (Moonlight White, 512 GB)","₹44,999","₹35,498",12 GB RAM,512 GB ROM,17.02 cm (6.7 inch) Full HD+ Super AMOLED Display,50MP + 50MP + 12MP,50MP Front Camera,5200 mAh Battery,Snapdragon 7 Gen 3 Processor
981,"LAVA Blaze (Glass Blue, 64 GB)",NaN,"₹9,699",3 GB RAM,64 GB ROM,16.51 cm (6.5 inch) HD+ Display,13MP Triple AI Camera + 8MP front camera,8MP Front Camera,5000 mAh Lithium Polymer Battery,Mediatek Helio A22 Processor
982,"vivo Y22 (Starlit Blue, 128 GB)","₹19,990","₹15,499",6 GB RAM,128 GB ROM,16.64 cm (6.55 inch) HD+ Display,50MP + 2MP,8MP Front Camera,5000 mAh Lithium Battery,Mediatek Helio G70 Processor
